# Imports & Setup

In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-7a3b  GCE       5                                       RUNNING  us-central1-a


In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
import builtins
import math
from google.cloud import storage
from collections import defaultdict
from contextlib import closing
import hashlib
import json
from inverted_index_gcp import InvertedIndex
from inverted_index_gcp import MultiFileReader
TUPLE_SIZE = 6
client = storage.Client()
nltk.download('stopwords')
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
full_path = "gs://wikidata_preprocessed/*"
corpus = spark.read.parquet(full_path)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

# Building an inverted index

In this part we bulid one function that bulid all the inverted indexes that we use for the information retrieval process.

In [ ]:
# All the function the userd in the process of bulid inverted index.

RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
stemmer = PorterStemmer()
all_stopwords = ['end','time','unit','use', 'known','new', 'was', 'how', 'for', 'could', 'please', 'then', 'whereupon', 'so', 'as', 'any', 'used', 'why', 'several', 'six', 'by', 'yourself', 'hence', 'did', 'whose', 'doing', 'n’t', 'show', 'her', 'yourselves', 'nothing', 'not', '‘ve', 'thereby', 'forty', 'each', 'whether', 'serious', 'perhaps', 'me', 'nevertheless', 'list', 'rather', 'elsewhere', 'former', 'seemed', 'up', 'than', 'except', 'via', 'can', 'everything', 'already', 'along', 'while', 'anyone', 'our', 'thus', 'must', 'either', 'move', '’ve', 'between', 'ever', 'does', 'whereafter', 'twelve', 'ca', 'in', 'again', 'itself', 'themselves', 'until', 'whence', 'she', 'us', 'say', 'no', 'amongst', 'were', 'whole', 'nine', 'under', 'everywhere', 'an', 'therein', 'go', 'would', 'beyond', 'become', 's', 'category', 'sixty', 'myself', 'they', 'about', 'since', 'put', 'amount', 'whatever', 'whereby', 'though', 'thru', 'a', "'ll", 'never', 'their', 'even', 'ours', 'twenty', 'am', 'someone', "'s", 'that', 'last', 'to', 'becoming', 'upon', 'one', 'this', 'meanwhile', 'more', 'else', 'himself', 'moreover', 'bottom', 'well', 'became', 'same', 'alone', '‘d', 'these', 'before', 'with', 'but', 'because', 'following', 'hers', 'really', 'thereupon', 'third', 'always', 'some', "'ve", 'hereby', 'his', 'above', 'thumb', 'part', 'few', 'formerly', "'d", '’d', 'however', 'history', 'seeming', 'having', 'somewhere', 'him', 'neither', 'do', 'whereas', 'besides', 'wherein', 'next', 'throughout', 'against', 'is', 'unless', 'yet', 'therefore', 'many', 'eight', 'still', 'name', 'nobody', 'top', 'mine', 'somehow', 'you', 'been', 'fifty', 'where', 'sometimes', 'get', 'hereafter', 'three', 'quite', 'towards', 'made', 'may', 'side', 'thereafter', 'beside', 'noone', 'none', 'be', 'anywhere', 'further', '‘ll', 'links', 'toward', 'due', 'my', 'what', 'references', 'beforehand', 'there', 'thence', 'together', 'four', 'among', 'at', "n't", 'such', 't', 'external', 'people', 'using', 'something', 'ourselves', 'keep', 'another', 'onto', 'latterly', '‘s', 'had', 'or', 'only', 'hundred', 'whenever', 'also', 'see', '‘re', 'whither', 'less', 'much', 'others', 'should', 'sometime', 'although', 'around', 'seem', 'of', 'down', 'five', 'them', 'on', '‘m', 'yours', 'very', 'various', 'empty', 'through', 'wherever', 'most', 'cannot', 'here', 'namely', 'mostly', 'per', 'during', 'whoever', 'out', 'we', 'second', '’re', 'both', 're', 'ten', 'don', 'anyhow', 'after', 'back', 'everyone', 'done', 'often', 'call', 'latter', 'seems', 'might', 'now', 'take', 'fifteen', 'every', '’m', 'own', 'becomes', 'all', 'make', 'which', 'full', 'indeed', 'nor', 'give', 'other', 'almost', 'hereupon', 'who', 'nowhere', '’s', 'including', "'re", 'too', 'i', 'he', 'its', 'those', 'the', 'it', 'first', 'herein', '’ll', 'just', 'over', 'have', 'and', "'m", 'when', 'eleven', 'below', 'if', 'theirs', 'behind', 'whom', 'are', 'will', 'from', 'into', 'enough', 'your', 'least', 'anyway', 'n‘t', 'herself', 'has', 'within', 'once', 'front', 'anything', 'afterwards', 'being', 'off', 'across', 'two', 'otherwise', 'regarding', 'without']

# Convert word to number
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

# Tokenize the each text / title and remove stop word. If ngram = True, concatenate adjacent words.
def get_tokens(text, ngram=False):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = [stemmer.stem(term) for term in tokens if term not in all_stopwords]
    if not ngram: return tokens
    if len(tokens)<2: return []
    b = Counter([''.join(x) for x in zip(tokens[0::1], tokens[1::1])])
    return [i[0] for i in b.most_common()][:10]

# Count number of apperance of word in text.
def word_count(text, id, ngram=False):
    tokens = get_tokens(text, ngram)
    words_counter = Counter(tokens)
    return [(word,(id,words_counter[word])) for word in words_counter]

# Count the number of word in text.
def len_doc(text, id, ngram=False):
    tokens = get_tokens(text, ngram)
    return (id, len(tokens))

# Calculate the size in the TD-IDF dimension.
def size_doc(text, id ,df ,corpus_size, ngram=False):
    tokens = get_tokens(text, ngram)
    words_counter = Counter(tokens)
    size = builtins.sum([(c/len(tokens)*math.log2(corpus_size/df[w]))**2 for w,c in words_counter.items() if w in df])
    return (id, math.sqrt(size))

# Sorting posting list, key is the term frequency in each doc.
def reduce_word_counts(unsorted_pl):
    return sorted(unsorted_pl, key = lambda x: x[1], reverse = True)

# Hash function the convert each token to bucket
def token2bucket_id(token, NUM_BUCKETS):
    return int(_hash(token),16) % NUM_BUCKETS

# Write postind lists to buckets.
def partition_postings_and_write(postings, bucket_name, NUM_BUCKETS):
    ps = postings.map(lambda x: (token2bucket_id(x[0], NUM_BUCKETS),[x]))
    ps = ps.reduceByKey(lambda x, y: x+y)
    ps = ps.mapValues(reduce_PLs)
    return ps.map(lambda x: InvertedIndex.write_a_posting_list((x[0], x[1]), bucket_name))

# Sort posting list for each bucket by the size of the posting list.
def reduce_PLs(unsorted_pl):
    sorted_Pl =  sorted(unsorted_pl, key = lambda x: len(x[1]), reverse = True)
    return [(i[0],i[1][:200]) for i in sorted_Pl]

# Calculate doc frequency of each word.
def calculate_df(postings):
    return postings.reduceByKey(lambda posting_list1, posting_list2: posting_list1+posting_list2)\
                .mapValues(lambda x: len(x))

In [ ]:
# Main function that creat each inverted index.
# params :
# corpus : wikipedia corpus
# bucket_name : the name of the bucket to store the inverted index
# NUM_BUCKETS : num of bins in the bucket
# filter_size : filter size for rare words
# ngram : bool to do the inverdet bgram or unigram
def Create_Inverted_Index(corpus, bucket_name, column, NUM_BUCKETS,  filter_size = 0, ngram=False):
  doc_pairs = corpus.select(column, "id").rdd
  word_counts = doc_pairs.flatMap(lambda x: word_count(x[0], x[1],ngram))
  postings = word_counts.groupByKey().mapValues(reduce_word_counts)
  # postings = postings.sortBy(lambda a: -len(a[1])).take(2000000)
  postings_filtered = postings.filter(lambda x: len(x[1])>filter_size and len(x[1])<900000)
  posting_locs_list = partition_postings_and_write(postings_filtered, bucket_name, NUM_BUCKETS).collect()
  w2df = calculate_df(postings)
#   id_len = doc_pairs.map(lambda x: len_doc(x[0], x[1],ngram))
  super_posting_locs = defaultdict(list)
  for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
    if not blob.name.endswith("pickle"):
      continue
    with blob.open("rb") as f:
      posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
       super_posting_locs[k].extend(v)
  inverted = InvertedIndex(bucket_name)
  inverted.posting_locs = super_posting_locs
  w2df_dict = w2df.collectAsMap()
  inverted.df = w2df_dict
#   inverted.DL = dict(id_len.collect())
#   id_size = doc_pairs.map(lambda x: size_doc(x[0], x[1], w2df_dict, len(inverted.DL), ngram))
#   inverted.DS = dict(id_size.collect())
  inverted.write_index('.', f'index_{bucket_name}')
  index_src = f"index_{bucket_name}.pkl"
  index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
  !gsutil cp $index_src $index_dst

In [ ]:
#Create inverted indexes
Create_Inverted_Index(corpus, 'title_inverted_index', 'title', 124, filter_size = 0)
Create_Inverted_Index(corpus, 'body_inverted_index', 'text', 248, filter_size = 70) 
Create_Inverted_Index(corpus, 'title2_inverted_index', 'title', 124, filter_size = 0, ngram=True)
Create_Inverted_Index(corpus, 'body2_inverted_index', 'text', 124, filter_size = 50, ngram=True)
Create_Inverted_Index(corpus, 'simple_title_inverted_index', 'title', 124, filter_size = 50)
Create_Inverted_Index(corpus, 'simple_body_inverted_index', 'text', 124, filter_size = 50)


# Create Page rank

In this section we build the Page Rank of the corpus.

In [ ]:
def generate_graph(pages):
  edges = pages.flatMap(lambda x: map(lambda y: (x[0], y[0]), x[1])).distinct()
  vertices = edges.flatMap(lambda x: x).distinct().map(lambda x:(x,))
  return edges, vertices

In [ ]:
pages_links = corpus.select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())

In [ ]:
# Store the page rank in bucket
pr.repartition(1).write.csv('gs://title_inverted_index/pr', compression="gzip")


# Anchor inverted index

In this section we create the anchor inverted index

In [ ]:
bucket_name = 'anchor_bucket'
anchor_InvertedIndex = InvertedIndex(bucket_name)

In [ ]:
pages_titles = corpus(full_path).select("id", 'title').rdd
pages_links = corpus(full_path).select("anchor_text").rdd
pg = pages_links.flatMap(lambda x:[(get_tokens(text), id) for id, text in x[0]])
postings = pg.flatMap(lambda x: [(y,x[1]) for  y in x[0]]).groupByKey()
anchor_posting = postings.mapValues(list).map(lambda x: (x[0], Counter(x[1]).most_common()))
postings_filtered = anchor_posting.filter(lambda x: len(x[1]) > 50).mapValues(lambda x: x[:50])
anchor_InvertedIndex.DL = pages_titles.map(lambda x: (x[0], x[1])).collectAsMap()
anchor_InvertedIndex.df = Counter(calculate_df(postings_filtered).collectAsMap())
posting_locs_list = partition_postings_and_write(postings_filtered, bucket_name, 124).collect()

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs_text = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_text[k].extend(v)
anchor_InvertedIndex.posting_locs = super_posting_locs_text

In [ ]:
# write the global stats out
anchor_InvertedIndex.write_index('.', f'index_{bucket_name}')
# upload to gs
inverted.write_index('.', f'index_{bucket_name}')
index_src = f"index_{bucket_name}.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst